In [3]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from nltk.corpus import stopwords
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

2022-05-12 23:04:59.530110: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-12 23:04:59.530216: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
train = pd.read_csv('tass2020_emotion_train.csv')
train.head()

,id,tweet,label
0,1,El Atlético resignado a perder HASHTAG 😔 http...,sadness
1,2,Leer proporciona una mejor visión del mundo 🤓 ...,joy
2,3,Amo a Arya Stark por encima de todas las cosas...,joy
3,4,Gracias HASHTAG es increíble que una niña logr...,others
4,5,Solo siento que hayamos perdido 24 escaños de ...,sadness


In [6]:
train['label'].unique()

array(['sadness ', 'joy ', 'others', 'others ', 'surprise ', 'disgust ',
       'anger ', 'fear '], dtype=object)

In [7]:
train.groupby('label').nunique()

,id,tweet
label,,
anger,600,600
disgust,113,113
fear,67,67
joy,1270,1270
others,1,1
others,2888,2886
sadness,706,706
surprise,241,241


In [8]:
train = train[['tweet','label']]

In [10]:
train["tweet"].isnull().sum() #No hay valores nulos

0

In [11]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

temp = []
#Splitting pd.Series to list
data_to_list = train['tweet'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['El Atlético resignado a perder HASHTAG 😔 ',
 'Leer proporciona una mejor visión del mundo 🤓 ¡Feliz HASHTAG y HASHTAG! 📖 ',
 'Amo a Arya Stark por encima de todas las cosas 😍😍😍#gameofthrones HASHTAG',
 'Gracias HASHTAG es increíble que una niña logre liderar tan imperativa movilización y supere a cualquier adulto en nivel de conciencia. Vergüenza deberíamos sentir los demás 😓#CambioClimático HASHTAG HASHTAG HASHTAG 🙏🌎💚',
 'Solo siento que hayamos perdido 24 escaños de cordura HASHTAG']

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(temp))

print(data_words[:10])

[['el', 'atletico', 'resignado', 'perder', 'hashtag'], ['leer', 'proporciona', 'una', 'mejor', 'vision', 'del', 'mundo', 'feliz', 'hashtag', 'hashtag'], ['amo', 'arya', 'stark', 'por', 'encima', 'de', 'todas', 'las', 'cosas', 'gameofthrones', 'hashtag'], ['gracias', 'hashtag', 'es', 'increible', 'que', 'una', 'nina', 'logre', 'liderar', 'tan', 'imperativa', 'movilizacion', 'supere', 'cualquier', 'adulto', 'en', 'nivel', 'de', 'conciencia', 'verguenza', 'deberiamos', 'sentir', 'los', 'demas', 'cambioclimatico', 'hashtag', 'hashtag', 'hashtag'], ['solo', 'siento', 'que', 'hayamos', 'perdido', 'escanos', 'de', 'cordura', 'hashtag'], ['solo', 'con', 'ver', 'con', 'la', 'intensidad', 'que', 'agitan', 'las', 'banderas', 'se', 've', 'que', 'no', 'han', 'quedado', 'satisfechos', 'los', 'de', 'user', 'sinceramente', 'me', 'alegro', 'muchisimo', 'hashtag', 'hashtag'], ['una', 'vez', 'mas', 'uno', 'de', 'los', 'mejores', 'jugadores', 'de', 'la', 'historia', 'leo', 'messi', 'al', 'al', 'en', 'hash

In [13]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))
print(data[:5])

['el atletico resignado perder hashtag', 'leer proporciona una mejor vision del mundo feliz hashtag hashtag', 'amo arya stark por encima de todas las cosas gameofthrones hashtag', 'gracias hashtag es increible que una nina logre liderar tan imperativa movilizacion supere cualquier adulto en nivel de conciencia verguenza deberiamos sentir los demas cambioclimatico hashtag hashtag hashtag', 'solo siento que hayamos perdido escanos de cordura hashtag']


In [14]:
data = np.array(data)

In [69]:
labels = np.array(train['label'])
y = []
for i in labels:
    if i == 'anger ':
        y.append(0)
    if i == 'disgust ':
        y.append(1)
    if i == 'fear ':
        y.append(2)
    if i == 'joy ':
        y.append(3)
    if i == 'others ':
        y.append(4)
    if i == 'others':
        y.append(4)
    if i == 'sadness ':
        y.append(5)
    if i == 'surprise ':
        y.append(6)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 7,dtype="float32")
del y

In [70]:
len(labels)

5886

In [71]:
from keras.models import Sequential
from keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  700  360    1]
 [   0    0    0 ...   65    1    1]
 [   0    0    0 ...  182 3571    1]
 ...
 [   0    0    0 ...    1   10  568]
 [   0    0    0 ...    1    1    1]
 [   0    0    0 ...   48    1    1]]


In [72]:
print(labels)

[[0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [73]:
X_train, X_test, y_train, y_test = train_test_split(tweets,labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

4414 1472 4414 1472


In [75]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20))
model1.add(layers.LSTM(15,dropout=0.5))
model1.add(layers.Dense(7,activation='softmax'))


model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=70,validation_data=(X_test, y_test),callbacks=[checkpoint1])

Epoch 1/70
138/138 [==============================] - ETA: 0s - loss: 1.5212 - accuracy: 0.4844
Epoch 1: val_accuracy improved from -inf to 0.48709, saving model to best_model1.hdf5
138/138 [==============================] - 23s 126ms/step - loss: 1.5212 - accuracy: 0.4844 - val_loss: 1.4343 - val_accuracy: 0.4871
Epoch 2/70
138/138 [==============================] - ETA: 0s - loss: 1.3803 - accuracy: 0.4941
Epoch 2: val_accuracy improved from 0.48709 to 0.49321, saving model to best_model1.hdf5
138/138 [==============================] - 16s 114ms/step - loss: 1.3803 - accuracy: 0.4941 - val_loss: 1.3784 - val_accuracy: 0.4932
Epoch 3/70
138/138 [==============================] - ETA: 0s - loss: 1.3267 - accuracy: 0.5109
Epoch 3: val_accuracy improved from 0.49321 to 0.51223, saving model to best_model1.hdf5
138/138 [==============================] - 16s 114ms/step - loss: 1.3267 - accuracy: 0.5109 - val_loss: 1.3396 - val_accuracy: 0.5122
Epoch 4/70
138/138 [==========================

Epoch 28/70
138/138 [==============================] - ETA: 0s - loss: 0.6446 - accuracy: 0.7873
Epoch 28: val_accuracy did not improve from 0.59443
138/138 [==============================] - 17s 123ms/step - loss: 0.6446 - accuracy: 0.7873 - val_loss: 1.3587 - val_accuracy: 0.5815
Epoch 29/70
138/138 [==============================] - ETA: 0s - loss: 0.6307 - accuracy: 0.7884
Epoch 29: val_accuracy did not improve from 0.59443
138/138 [==============================] - 16s 115ms/step - loss: 0.6307 - accuracy: 0.7884 - val_loss: 1.3426 - val_accuracy: 0.5781
Epoch 30/70
138/138 [==============================] - ETA: 0s - loss: 0.6220 - accuracy: 0.7923
Epoch 30: val_accuracy did not improve from 0.59443
138/138 [==============================] - 16s 114ms/step - loss: 0.6220 - accuracy: 0.7923 - val_loss: 1.3475 - val_accuracy: 0.5761
Epoch 31/70
138/138 [==============================] - ETA: 0s - loss: 0.6074 - accuracy: 0.8029
Epoch 31: val_accuracy did not improve from 0.59443
13

Epoch 57/70
138/138 [==============================] - ETA: 0s - loss: 0.4366 - accuracy: 0.8514
Epoch 57: val_accuracy did not improve from 0.59443
138/138 [==============================] - 16s 112ms/step - loss: 0.4366 - accuracy: 0.8514 - val_loss: 1.5669 - val_accuracy: 0.5598
Epoch 58/70
138/138 [==============================] - ETA: 0s - loss: 0.4300 - accuracy: 0.8575
Epoch 58: val_accuracy did not improve from 0.59443
138/138 [==============================] - 15s 112ms/step - loss: 0.4300 - accuracy: 0.8575 - val_loss: 1.5708 - val_accuracy: 0.5591
Epoch 59/70
138/138 [==============================] - ETA: 0s - loss: 0.4278 - accuracy: 0.8568
Epoch 59: val_accuracy did not improve from 0.59443
138/138 [==============================] - 16s 112ms/step - loss: 0.4278 - accuracy: 0.8568 - val_loss: 1.5914 - val_accuracy: 0.5401
Epoch 60/70
138/138 [==============================] - ETA: 0s - loss: 0.4353 - accuracy: 0.8573
Epoch 60: val_accuracy did not improve from 0.59443
13